In [19]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_score

In [20]:
# Загрузите данные из файла CSV
files: list[str] = [
    "dataset/annotation/hr_bot_noise.json",
    "dataset/annotation/hr_bot_clear.json",
    "dataset/annotation/hr_bot_synt.json",
    "dataset/annotation/luga.json",
]

li = []
for filename in files:
    df = pd.read_json(filename)
    li.append(df)
data = pd.concat(li, axis=0, ignore_index=True)
data = pd.read_csv("dataset/transcribed.csv")
data.head()

,Unnamed: 0,audio_filepath,id,text,label,attribute,transcription
0,48,dataset/hr_bot_noise/50a3b202-76fe-11ee-bbaa-c...,50a3b202-76fe-11ee-bbaa-c09bf4619c03,осадить на девять вагонов,4,9,вас садите на девять вагонов
1,1086,dataset/hr_bot_clear/65ecbc1d-76ff-11ee-87b1-c...,65ecbc1d-76ff-11ee-87b1-c09bf4619c03,продолжаем осаживание,5,-1,продолжаем осаживание
2,1208,dataset/hr_bot_clear/39039b4f-76ff-11ee-9039-c...,39039b4f-76ff-11ee-9039-c09bf4619c03,протянуть на три вагона,10,3,протянуть на три вагон
3,2591,dataset/hr_bot_clear/44adf89b-76ff-11ee-8943-c...,44adf89b-76ff-11ee-8943-c09bf4619c03,отцепка,11,-1,отцепка
4,11,dataset/hr_bot_noise/6a08ef79-76fe-11ee-9305-c...,6a08ef79-76fe-11ee-9305-c09bf4619c03,назад на башмак,12,-1,назад на башмак


In [21]:
commands = data["attribute"]
queries = data["transcription"]

# Предварительная обработка текста (например, удаление стоп-слов и стемминг)
vectorizer = TfidfVectorizer(
    lowercase=True, sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english"
)
classifier = MultinomialNB()

pipeline = Pipeline(
    [
        ("tfidf", vectorizer),
        ("classifier", classifier),
    ]
)

In [22]:
# Разделите данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(queries, commands, test_size=0.2, random_state=42)

# Обучите модель наивного байесовского классификатора
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.5, min_df=5, stop_words='english',
                                 sublinear_tf=True)),
                ('classifier', MultinomialNB())])

In [23]:
score = pipeline.score(X_test, y_test)
print('Точность на тестовом наборе:', score)

scores = cross_val_score(pipeline, queries, commands, cv=5)
print('Средняя точность по перекрестной проверке:', scores.mean())

Точность на тестовом наборе: 0.8888888888888888
Средняя точность по перекрестной проверке: 0.872316384180791


In [24]:
import joblib

joblib.dump(pipeline, "text2attr.pkl")

['text2attr.pkl']